SUMMARY: Will find all of the multi instance videos in the PairedTestingSessions directory that don't have a corresponding predictions file and then creates a job txt file that will save to the misha jobs folder

In [1]:
NUM_BATCH_JOBS = 'training_coop_track' 
mb_per_sec = 0.20447582936507938 # with partition=gpu, cpus-per-gpu=8, mem-per-cpu=10G, gpus=a40:1

In [2]:
import os
import random

In [3]:
defaultdir = '/gpfs/radev/pi/saxena/aj764'
rootdir = f'{defaultdir}/Training_COOPERATION/'

Compiles a list of all of the sessions in PairedTestingSessions that have a Videos folder in them

In [4]:
vid_subdirs = []
for subdir, dirs, files in os.walk(rootdir):
    vid_subdirs.append(subdir)
vid_subdirs = sorted(vid_subdirs)

In [5]:
multi_vids = {}
for vids in vid_subdirs:
    files = os.listdir(vids)
    cut_vids = vids[28:]
    multi_vids[cut_vids] = []
    for file in files:
        if file.endswith('.mp4'): # and int(file[:2]) >= 4:
            multi_vids[cut_vids].append(file)
            

In [6]:
len_tot = 0
for key, value in multi_vids.items():
    len_tot += len(value)
print(f'There are {len_tot} multi instance videos')

There are 212 multi instance videos


In [7]:
# split the multi-instance videos into their respective color pairs...
color_vids = {}
for key, value in multi_vids.items():
    for vid in value:
        parsed = vid.split('-')
        
        trial_color = [parsed[0][-1], parsed[1][5]]
        trial_key = ''
        if 'R' in trial_color:
            trial_key += 'R'
        if 'G' in trial_color:
            trial_key += 'G'
        if 'Y' in trial_color:
            trial_key += 'Y'
        if 'B' in trial_color:
            trial_key += 'B'
        if trial_key not in color_vids.keys():
            color_vids[trial_key] = []
        color_vids[trial_key].append(vid)

In [8]:
len_tot = 0
for key, value in color_vids.items():
    print(f'There are {len(value)} videos from {key} color pair')
    len_tot += len(value)
print('\n')
print(f'There are {len_tot} multi instance videos')

There are 69 videos from YB color pair
There are 18 videos from GB color pair
There are 28 videos from RG color pair
There are 19 videos from GY color pair
There are 32 videos from RB color pair
There are 46 videos from RY color pair


There are 212 multi instance videos


Creates a string composed of all of the necessary commands to run inference on all multi instance videos for a particular color that don't have a corresponding predictions file.

In [34]:
start_command = f'module load miniconda; conda activate sleap; cd {defaultdir};'
centroid_model = 'Tracking/DLC_Pairs/YellowBlue-V1-2024-07-23/SLEAP/models/250202_164327.centroid'
topdown_model = 'Tracking/DLC_Pairs/YellowBlue-V1-2024-07-23/SLEAP/models/250202_164327.multi_class_topdown'
color_pair = 'YB' # chose from one of these pairs ['RY', 'YB', 'GB', 'GY', 'RB', 'RG']

In [52]:
command_lines = ''
for i, session in enumerate(multi_vids.keys()): 
    video_list = multi_vids[session]
    output_path = defaultdir + '/' + session + '/Tracking'

    # makes directory for tracking output if not already made
    if not os.path.isdir(output_path):
        os.mkdir(output_path)
    if not os.path.isdir(output_path + '/slp'):
        os.mkdir(output_path + '/slp')
    if not os.path.isdir(output_path + '/h5'):
        os.mkdir(output_path + '/h5')

    # adds command for every video that doesn't have a corresponding prediction
    for video in video_list:
        if video in color_vids[color_pair]:
            video_path = f'{session}/{video}'
            output_file = video[:-3] + 'predictions.'
            if not os.path.isfile(output_path + '/slp/' + output_file + 'slp'):
                track_command = f'sleap-track "{video_path}" --first-gpu -o "{output_path + '/slp/' + output_file + 'slp'}" -m "{centroid_model}" -m "{topdown_model}"'
                convert_command = f'; sleap-convert --format analysis -o "{output_path + '/h5/' + output_file + 'h5'}" "{output_path + '/slp/' + output_file + 'slp'}"'
                command_lines += (start_command + track_command + convert_command + '\n')

Saves the commands to a txt file in the job directory.

In [53]:
jobdir = '/gpfs/radev/project/saxena/aj764/ood/projects/default/'

In [54]:
commanddir = jobdir + str(NUM_BATCH_JOBS)
if not os.path.isdir(commanddir): 
    os.mkdir(commanddir)
with open(f"{commanddir}/{color_pair}_vids_job.txt", "w") as file:
    file.write(command_lines) 

Keeps track of which videos have predictions and which don't

In [12]:
done_vids = []
done_vids_size = []
not_done_vids = []
not_done_vids_size = []
for i, session in enumerate(multi_vids.keys()):
    video_list = multi_vids[session]
    output_path = defaultdir + '/' + session + '/Tracking'
        
    for video in video_list:
        video_path = f'{session}/{video}'
        output_file = video[:-3] + 'predictions.'
        if os.path.isfile(output_path + '/slp/' + output_file + 'slp') and os.path.isfile(output_path + '/h5/' + output_file + 'h5'):
            done_vids.append(session + '/' + video)
            done_vids_size.append((os.path.getsize(defaultdir + '/' + session + '/' + video)) / 10 ** 6)
        else:
            not_done_vids.append(session + '/' + video)
            not_done_vids_size.append((os.path.getsize(defaultdir + '/' + session + '/' + video)) / 10 ** 6)
num_done = len(done_vids)
num_not_done = len(not_done_vids)
time_done = sum(done_vids_size) / mb_per_sec
time_not_done = sum(not_done_vids_size) / mb_per_sec

In [13]:
def seconds(secs):
    days = secs//86400
    hours = (secs - days*86400)//3600
    minutes = (secs - days*86400 - hours*3600)//60
    seconds = secs - days*86400 - hours*3600 - minutes*60
    result = ("{0} day{1}, ".format(days, "s" if days!=1 else "") if days else "") + \
    ("{0} hour{1}, ".format(hours, "s" if hours!=1 else "") if hours else "") + \
    ("{0} minute{1}, ".format(minutes, "s" if minutes!=1 else "") if minutes else "") + \
    ("{0} second{1}, ".format(seconds, "s" if seconds!=1 else "") if seconds else "")
    return result[:-2]

In [14]:
print(f'{round((len(done_vids) / (len(not_done_vids)+1)) * 100, 2)}% of videos have been tracked ({len(done_vids)} tracked videos, {len(not_done_vids)} untracked videos)')   
print(f'compute time of tracked videos: {seconds(time_done)}')
print(f'compute time of untracked videos: {seconds(time_not_done)}')
print(f'realish time of tracked videos: {seconds(time_done / 4)}')
print(f'realish time of untracked videos: {seconds(time_not_done / 4)}')

46.9% of videos have been tracked (68 tracked videos, 144 untracked videos)
compute time of tracked videos: 5.0 days, 13.0 hours, 13.0 minutes, 4.423740928410552 seconds
compute time of untracked videos: 12.0 days, 6.0 hours, 1.0 minute, 1.6412856185343117 seconds
realish time of tracked videos: 1.0 day, 9.0 hours, 18.0 minutes, 16.105935232102638 seconds
realish time of untracked videos: 3.0 days, 1.0 hour, 30.0 minutes, 15.410321404633578 seconds


In [17]:
random.choice(done_vids)

'Training_COOPERATION/070524/070524_COOPTRAIN_LARGEARENA_HF003B-HF004Y_Camera2.mp4'

In [19]:
len('Training_COOPERATION/070524/')

28

In [22]:
color_pair = 'YB'
match = False
# for vid in color_vids[color_pair]: 
for vid in color_vids[color_pair]:
    match = False
    for second_vid in done_vids:
        if second_vid[28:] == vid:
            match = True
    if not match:
        print(vid)
    

041524_COOPTRAIN_LARGEARENA_EB009B-EB019Y_Camera2.mp4
